In [13]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter
import numpy as np
from sklearn.linear_model import LogisticRegression
from statsmodels.discrete.discrete_model import Logit
from statsmodels.tools.tools import add_constant
from sklearn.linear_model import SGDClassifier
import random

In [3]:
#Load Data
features = [i.replace(' ', '_') for i in load_breast_cancer().feature_names.tolist()]

breast_cancer_df = pd.DataFrame(load_breast_cancer().data,columns=features)
target_df = pd.DataFrame(load_breast_cancer().target, columns=['y'])

df = pd.concat([target_df,breast_cancer_df],axis=1)

In [30]:
#Create small data set and rare event data set
num_successes_for_rare = int(((((1-df.y.mean())*df.shape[0])/.95)-((1-df.y.mean())*df.shape[0]))//1)
rare_inds = sorted(list(df[df.y==0].index) + random.sample(list(df[df.y==1].index),num_successes_for_rare))
small_inds = random.sample(sorted(list(df.index)),50)
rare_df = df.iloc[rare_inds,:]
small_df = df.iloc[small_inds,:]

In [88]:
def DAP(df,y_var_name):
    '''Perform log-f(1,1) data augmentation
       Returns augmented df and observation weights'''
    
    num_rows = 2*(df.shape[1]-1)
    y_ind = df.columns.get_loc(y_var_name)
    
    aug = pd.DataFrame(0,columns=df.columns,index=(range(num_rows)))
    
    #augment y variable
    aug.iloc[range(0,num_rows,2),y_ind]=1
    y = aug[y_var_name]
    
    #augment X variables
    X = aug.drop(y_var_name,axis=1)
    for ind, rows in enumerate(range(0,X.shape[0],2)):
         X.iloc[rows:rows+2,ind]=1
    
    #bring it all together
    aug = pd.concat([y,X],axis=1)
    f_df = df.append(aug)
    
    #add offset
    f_df['real_data']=1
    f_df['real_data'][-aug.shape[0]:]=0
    
    #Calculate weights
    weights = f_df['real_data'].apply(lambda x: 0.5 if x == 0 else 1)
    return f_df, weights      

In [90]:
all_aug, all_weights = DAP(df,'y')
small_aug, small_weights = DAP(small_df,'y')
rare_aug, rare_weights = DAP(rare_df,'y')

/Users/remycanario/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [92]:
#Convert data to R
with localconverter(ro.default_converter + pandas2ri.converter):
    #for firth
    firth_all_r = ro.conversion.py2ri(df)
    firth_small_r = ro.conversion.py2ri(small_df)
    firth_rare_r = ro.conversion.py2ri(rare_df)
    
    #for logf
    logf_all_r = ro.conversion.py2ri(all_aug)
    all_weights_r = ro.vectors.FloatVector(all_weights)
    logf_small_r = ro.conversion.py2ri(small_aug)
    small_weights_r = ro.vectors.FloatVector(small_weights)
    logf_rare_r = ro.conversion.py2ri(rare_aug)
    rare_weights_r = ro.vectors.FloatVector(rare_weights)

In [93]:
base = importr('base')
d = {'package.dependencies': 'package_dot_dependencies',
     'package_dependencies': 'package_uscore_dependencies'}
brglm = importr('brglm',robject_translations=d)

In [94]:
formula = 'y ~ ' + " + ".join(features)

In [131]:
len(features)

30

In [109]:
model = brglm.brglm(formula, data = firth_small_r, family='binomial',pl=True)
small_summary = base.summary(model)
small_summary_dic = {}
for i in range(len(small_summary.names)):
    try:
        small_summary_dic[small_summary.names[i]]=pandas2ri.converter.ri2py(list(small_summary)[i])
    except:
        pass

small_coefs = pd.DataFrame(small_summary_dic['coefficients'],columns=(['Coef','SE','Z','P']),index=df.columns)
small_preds_r = ro.r.predict(model,firth_all_r)

In [146]:
def get_r_firth_results(df, test_df, formula):
    model = brglm.brglm(formula, data = df, family='binomial',pl=True)
    summary = base.summary(model)
    summary_dic = {}
    for i in range(len(summary.names)):
        try:
            summary_dic[summary.names[i]]=pandas2ri.converter.ri2py(list(summary)[i])
        except:
            pass
    summary_dic['coefficients']
    coefs = pd.DataFrame(summary_dic['coefficients'],columns=(['Coef','SE','Z','P']))
    preds = ro.r.predict(model,firth_all_r)
    preds = [i for i in preds]                   
    return preds, coefs

In [147]:
small_pred, small_coef = get_r_results(firth_small_r,firth_all_r,formula)
small_pred = [i for i in small_pred]

In [150]:
def get_r_logf_results(df, test_df, formula, weights):
    model = brglm.brglm(formula, data = df, family='binomial',pl=True)
    summary = base.summary(model)
    summary_dic = {}
    for i in range(len(summary.names)):
        try:
            summary_dic[summary.names[i]]=pandas2ri.converter.ri2py(list(summary)[i])
        except:
            pass
    summary_dic['coefficients']
    coefs = pd.DataFrame(summary_dic['coefficients'],columns=(['Coef','SE','Z','P']))
    preds = ro.r.predict(model,firth_all_r)
    preds = [i for i in preds]                   
    return preds, coefs

In [137]:
rare_pred, rare_coef = get_r_results(firth_rare_r,firth_all_r,formula)
rare_pred = [i for i in rare_pred]

30

[-1.475788243558697,
 0.6769973145097323,
 -1.1673150719842842,
 -3.5620285961755003,
 -0.8284021775683528,
 -0.9892561641397244,
 -0.4850712968263537,
 0.6785641340185167,
 -0.33762879867674017,
 -0.6693177413150999,
 0.5487717760483358,
 -0.6784468956048406,
 1.810223878829877,
 0.6254505980693285,
 0.932227845722538,
 -2.0170584590119347,
 -0.4754171759761722,
 -0.7572136455692364,
 -0.32816310192870013,
 1.6636364055843278,
 1.2266329921314998,
 1.6798722850760095,
 -0.6496937373204418,
 0.2774433901010931,
 -2.105124077064894,
 -1.271236001724579,
 0.34471155828809463,
 -0.6025727855736827,
 -3.4190881099621837,
 1.015780295997338,
 -2.29256195330972,
 -4.4802950044992835,
 -1.4263778677060286,
 -2.070056116693159,
 -1.035076317845689,
 -1.8784918460936832,
 -0.17723123547303907,
 1.2187654759119342,
 1.701170826722003,
 1.629643076637503,
 1.2908359109426684,
 0.9690180082640536,
 -2.0190416170348304,
 -1.317434829062039,
 -0.6826899516165367,
 -1.698047581721619,
 1.495918372374